# evaluation

In [ ]:
import os, glob, pandas as pd
from textstat import flesch_reading_ease
from surprise import Dataset, Reader, SVD
from recbole.quick_start import run_recbole

## Gather summaries

In [ ]:
records = []
for idx, path in enumerate(sorted(glob.glob('evaluation/*.txt'))):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
    fre = flesch_reading_ease(text)         
    rating = max(1, min(5, round(fre/20)))  
    records.append(['critic', f'file{idx}', rating])

ratings = pd.DataFrame(records, columns=['user', 'item', 'rating'])

## Surprise – quick latent-factor view

In [ ]:
reader   = Reader(rating_scale=(1, 5))
data     = Dataset.load_from_df(ratings, reader)
trainset = data.build_full_trainset()
algo     = SVD(n_epochs=5, verbose=False).fit(trainset)

print('Surprise: first 5 predicted scores')
for _, row in ratings.head().iterrows():
    est = algo.predict(row.user, row.item).est
    print(f'{row.item:<10}  truth={row.rating}  predicted={est:0.2f}')


## RecBole – needs its own folder layout

In [ ]:
os.makedirs('dataset/summaries', exist_ok=True)
ratings.to_csv('dataset/summaries/summaries.inter',
               index=False, sep='\t', header=True)

# A few sensible defaults are already baked into quick_start
run_recbole(model='MF', dataset='summaries',
            config_dict={'epochs': 10,           
                         'eval_step': 0,         
                         'metric_decimal_place': 2})